In [1]:
import os
import shutil
from sahi.utils.coco import Coco, CocoCategory, CocoImage, CocoAnnotation
from sahi.utils.file import save_json
import json
from PIL import Image
import ast
import pandas as pd
import random

In [2]:
#https://medium.com/codable/convert-any-dataset-to-coco-object-detection-format-with-sahi-95349e1fe2b7

In [2]:
classes = ["X1-Y1-Z2", 
           "X1-Y2-Z1", 
           "X1-Y2-Z2", 
           "X1-Y2-Z2-CHAMFER",
           "X1-Y2-Z2-TWINFILLET", 
           "X1-Y3-Z2", 
           "X1-Y3-Z2-FILLET", 
           "X1-Y4-Z1", 
           "X1-Y4-Z2", 
           "X2-Y2-Z2", 
           "X2-Y2-Z2-FILLET"]

In [3]:
id_to_label={}
label_to_id={}
for i in range (len(classes)):
    id_to_label.update({i:classes[i]})
    label_to_id.update({classes[i]:i})
id_to_label

{0: 'X1-Y1-Z2',
 1: 'X1-Y2-Z1',
 2: 'X1-Y2-Z2',
 3: 'X1-Y2-Z2-CHAMFER',
 4: 'X1-Y2-Z2-TWINFILLET',
 5: 'X1-Y3-Z2',
 6: 'X1-Y3-Z2-FILLET',
 7: 'X1-Y4-Z1',
 8: 'X1-Y4-Z2',
 9: 'X2-Y2-Z2',
 10: 'X2-Y2-Z2-FILLET'}

In [5]:
coco = Coco()
coco

In [6]:
for key, value in id_to_label.items():
    coco.add_category(CocoCategory(id=key, name=value))

In [9]:
coco_train = Coco()
coco_train

In [7]:
coco_test = Coco()
coco_test

In [10]:
for key, value in id_to_label.items():
    coco_train.add_category(CocoCategory(id=key, name=value))
    coco_test.add_category(CocoCategory(id=key, name=value))

In [34]:
def split_and_create_coco_image(input_folder, class_name, class_id, output_path_train, output_path_test): #es gli do dentro X1Y1Z2
    # Iterate through all folders and subfolders in the dataset
    for folder, subfolders, files in os.walk(input_folder):
        # Process only leaf folders (folders without subfolders)
        if not subfolders:
            # Look for an image file (change the extension accordingly)
            image_files = [file for file in files if file.lower().endswith(('.jpeg'))]
            image_name = os.path.join(folder, image_files[0])
            print(f"Image path: {image_name}")

                # Look for a .txt file
            txt_files = [file for file in files if file.lower().endswith('.txt')]
            txt_file_path = os.path.join(folder, txt_files[0])
            with open(txt_file_path, 'r') as txt_file:
                string_content = txt_file.read()
            print(f"Text content: {string_content}")
            content_to_list = ast.literal_eval(string_content)
            print(f"{type(content_to_list)}, {content_to_list}")

            # max_value_x=max(content_to_list, key=lambda sublist: sublist[0])[0]
            # min_value_x=min(content_to_list, key=lambda sublist: sublist[0])[0]

            # max_value_y=max(content_to_list, key=lambda sublist: sublist[1])[1]
            # min_value_y=min(content_to_list, key=lambda sublist: sublist[1])[1]

            # max_value_z=max(content_to_list, key=lambda sublist: sublist[2])[2]
            # min_value_z=min(content_to_list, key=lambda sublist: sublist[2])[2]
            x_top_left= content_to_list[0]
            y_top_left=content_to_list[1]
            x_bottom_right= content_to_list[2]
            y_bottom_right = content_to_list[3]

            offset = 20

            height=abs(y_top_left-y_bottom_right)
            width= abs(x_bottom_right-x_top_left)

            width_tot, height_tot = Image.open(image_name).size
            print(f"h={height_tot}, w={width_tot}")

            only_image_name = os.path.basename(image_name)
            
            coco_image = CocoImage(file_name=only_image_name, height=height_tot, width=width_tot)


            coco_image.add_annotation(
                CocoAnnotation(
                bbox=[x_top_left, y_top_left, width, height],
                category_id=class_id,
                category_name=class_name
                )
            )

            random_number = random.randint(0, 3)
            if random_number==0:
                coco_test.add_image(coco_image)
                shutil.copy(image_name, output_path_test)
            else:
                coco_train.add_image(coco_image)
                shutil.copy(image_name, output_path_train)


In [35]:
for key, value in id_to_label.items():
    split_and_create_coco_image(f"./dataset_v5/{value}", value, key, "./train2", "./test2")

Image path: ./dataset_v5/X1-Y1-Z2/9-47/X1-Y1-Z2-9-47.jpeg
Text content: [489, 519, 525, 505]

<class 'list'>, [489, 519, 525, 505]
h=1024, w=1024
Image path: ./dataset_v5/X1-Y1-Z2/7-36/X1-Y1-Z2-7-36.jpeg
Text content: [506, 533, 527, 491]

<class 'list'>, [506, 533, 527, 491]
h=1024, w=1024
Image path: ./dataset_v5/X1-Y1-Z2/5-1/X1-Y1-Z2-5-1.jpeg
Text content: [496, 527, 520, 497]

<class 'list'>, [496, 527, 520, 497]
h=1024, w=1024
Image path: ./dataset_v5/X1-Y1-Z2/6-28/X1-Y1-Z2-6-28.jpeg
Text content: [502, 527, 521, 497]

<class 'list'>, [502, 527, 521, 497]
h=1024, w=1024
Image path: ./dataset_v5/X1-Y1-Z2/5-25/X1-Y1-Z2-5-25.jpeg
Text content: [476, 419, 553, 605]

<class 'list'>, [476, 419, 553, 605]
h=1024, w=1024
Image path: ./dataset_v5/X1-Y1-Z2/9-48/X1-Y1-Z2-9-48.jpeg
Text content: [456, 459, 515, 565]

<class 'list'>, [456, 459, 515, 565]
h=1024, w=1024
Image path: ./dataset_v5/X1-Y1-Z2/6-36/X1-Y1-Z2-6-36.jpeg
Text content: [506, 533, 527, 491]

<class 'list'>, [506, 533, 527, 

In [36]:
save_json(data=coco_train.json, save_path="./train2/coco_train2.json")
save_json(data=coco_test.json, save_path="./test2/coco_test2.json")

In [37]:
#couting
files_train = [f for f in os.listdir("./train2") if os.path.isfile(os.path.join("./train2", f))]
files_test = [f for f in os.listdir("./test2") if os.path.isfile(os.path.join("./test2", f))]
print(f"number of images in train:{len(files_train)}, number of images in test:{len(files_test)}")

number of images in train:381, number of images in test:121
